# Майнор по Анализу Данных, Группа ИАД-4
## 7/12/2017 Прогнозирование временных рядов

In [ ]:
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
import numpy as np
from itertools import product

plt.style.use('ggplot')
    
%matplotlib inline

plt.rcParams['figure.figsize'] = (14, 7)

In [ ]:
import statsmodels
statsmodels.__version__

**Должно быть что-то >= 0.8**, если нет - то обновите пакет (требуется перезагрузка kernel)

### Перейдем к данным

Известны [ежемесячные продажи австралийского вина](https://cloud.mail.ru/public/MUy3/UQKzvb8zt) в тысячах литров с января 1980 по июль 1995, необходимо построить прогноз на следующие три года.

In [ ]:
wine = pd.read_csv('monthly-australian-wine-sales.csv',',', index_col=['month'], parse_dates=['month'], dayfirst=True)

In [ ]:
wine.head()

In [ ]:
wine.sales.plot()
plt.ylabel('Wine sales')

Проверим гипотезу о стационарности ряда

In [ ]:
print "Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(wine.sales)[1] 

Ряд можно представить в виде трех компонент:
* $S_t$ - сезонная компонента
* $T_t$ - тренд
* $E_t$ - остаточная компонента

Аддитивное разложение ряда:
$$ y_t = S_t + T_t + E_t, $$

Мультипликативное разложение ряда:
$$ y_t = S_t \times T_t \times E_t, $$

**Вопрос**: как из мультипликативного разложения можно перейти в аддитивное?


Разложим наш ряд на эти компоненты - ниже вы увидите результат разложения методом [STL](http://www.wessa.net/download/stl.pdf).

In [ ]:
fig = sm.tsa.seasonal_decompose(wine.sales).plot()

Как соотносятся эти рисунки и результаты проверки на стационарность?

### Стабилизация дисперсии

Сделаем преобразование Бокса-Кокса для стабилизации дисперсии.

Это преобразование имеет параметр $\lambda$  и имеет следующий вид:

* $w_t = \log(y_t)$ если $\lambda = 0$
* $w_t = (y_t^\lambda -1)/\lambda$ иначе

Параметр $\lambda$ определяется автоматически при решении задачи максимизации [правдоподобия](http://www.jekyll.math.byuh.edu/papers/mspaper99.pdf).

In [ ]:
wine['sales_box'], lmbda = stats.boxcox(wine.sales)
wine.sales_box.plot()
plt.ylabel(u'Transformed wine sales')
print("Оптимальный параметр преобразования Бокса-Кокса: %f" % lmbda)
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(wine.sales_box)[1])

### Стационарность

Критерий Дики-Фуллера отвергает гипотезу нестационарности, но визуально в данных виден тренд. 

Попробуем сезонное дифференцирование; сделаем на продифференцированном ряде STL-декомпозицию и проверим стационарность:

In [ ]:
wine['sales_box_diff'] = wine.sales_box - wine.sales_box.shift(12)
fig = sm.tsa.seasonal_decompose(wine.sales_box_diff[12:]).plot()
print "Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(wine.sales_box_diff[12:])[1]

Критерий Дики-Фуллера не отвергает гипотезу нестационарности, и полностью избавиться от тренда не удалось. 

Попробуем добавить ещё обычное дифференцирование:

In [ ]:
wine['sales_box_diff2'] = wine.sales_box_diff - wine.sales_box_diff.shift(1)
fig = sm.tsa.seasonal_decompose(wine.sales_box_diff2[13:]).plot()   
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(wine.sales_box_diff2[13:])[1])

Гипотеза нестационарности отвергается, и визуально ряд выглядит лучше — тренда больше нет. 

## Подбор параметров модели

Посмотрим на ACF и PACF полученного ряда:

Вспомним, как считается выборочная автокорреляция (ACF)

$$r_\tau = \frac{\sum_{t=1}^{T-\tau} (y_t - \bar{y}) (y_{t+\tau} - \bar{y})}{\sum_{t=1}^{T} (y_t - \bar{y})^2}$$

PACF выполняет корректировку на возможную корреляцию между лагами меньшего порядка. <br\>
*Если есть корреляция между $y_{t}$ и $y_{t-1}$ (а соответственно между $y_{t-1}$ и $y_{t-2}$) то корреляция между $y_{t}$ и $y_{t-2}$ тоже будет высока*

In [ ]:
# fig, ax = plt.subplots(2,1)
fig = sm.graphics.tsa.plot_acf(wine.sales_box_diff2[13:].values.squeeze(), lags=48)

In [ ]:
fig = sm.graphics.tsa.plot_pacf(wine.sales_box_diff2[13:].values.squeeze(), lags=48)

### Начальное приближение

Начальные приближения: Q=1, q=2, P=1, p=4

* Q = 1, потому что на графике ACF наблюдается значимое значение на лаге 12 - это сезонность
* q = 2, потому что на графике ACF налюдается значимое значение на лаге 1 и 2
* P = 1, потому что на графике PACF наблюдается значимое значение на лаге 12 - это сезонность
* p = 2, потому что на графике PACF налюдается значимое значение на лаге 1 - 4

In [ ]:
D=1 # Потому что мы сезонный сдвиг
d=1 # Потому что мы сделали обычный сдвиг на 1 шаг

p = 2
q = 2
P = 1
Q = 1


model = sm.tsa.statespace.SARIMAX(wine.sales_box, order=(p, d, q), 
                                seasonal_order=(P, D, Q, 12)).fit(disp=-1)

[AIC](https://ru.wikipedia.org/wiki/%D0%98%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BA%D1%80%D0%B8%D1%82%D0%B5%D1%80%D0%B8%D0%B9_%D0%90%D0%BA%D0%B0%D0%B8%D0%BA%D0%B5) - величина, основанная на сложности и точности модели. Чем меньше значение, тем лучше модель.

In [ ]:
model.aic

In [ ]:
model.summary()

Анализ остатков

In [ ]:
_, ax = plt.subplots(2,1)
model.resid[13:].plot(ax=ax[0])
ax[0].set_ylabel(u'Residuals')

fig = sm.graphics.tsa.plot_acf(model.resid[13:].values.squeeze(), lags=48, ax=ax[1])

print "Критерий Стьюдента: p=%f" % stats.ttest_1samp(model.resid[13:].values, 0)[1]
print "Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(model.resid[13:])[1] 

Реализуйте фукнцию которая делает обратное преобразование Бокс-Кокса и выполните прогноз

In [ ]:
def invboxcox(y,lmbda):
    ## Your Code Here

In [ ]:
wine['model'] = invboxcox(model.fittedvalues, lmbda)
wine.sales.plot()
wine.model[13:].plot(color='b')
plt.ylabel('Wine sales')

### Перебор параметров

Задайте наборы параметров (p, q, P, Q) и выберите модель с наилучшим набором по критерию aic

In [ ]:
# Зададим распределение параметров и найдем лучший набор
ps = range(0, ?)
qs = range(0, ?)
Ps = range(0, ?)
Qs = range(0, ?)

parameters_list = ?

In [ ]:
# Your Code Here

In [ ]:
%%time
results = []
best_aic = np.inf
warnings.filterwarnings('ignore')

for param in parameters_list:
    #try except нужен, потому что на некоторых наборах параметров модель не обучается
    try:
        model=..
    except ValueError:
        print('wrong parameters:', param)
        continue
    #сохраняем лучшую модель, aic, параметры
    ## Your Code Here

In [ ]:
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending=True).head())

Лучшая модель:

In [ ]:
print(best_model.summary())

### Прогноз

In [ ]:
from pandas.tseries.offsets import MonthBegin

In [ ]:
best_model.predict(start=176, end=211)

In [ ]:
## Your Code Here

## Facebook Phophet

Facebook выложил собственную библиотеку автоматического прогнозирования временных рядов [Prophet](https://github.com/facebook/prophet).

Если у вас получится ее установить - можете поиграться